In [15]:
import keras

from keras.preprocessing.image import img_to_array
from keras.models import load_model

import cv2
import numpy as np
print("Your OpenCV version => {}".format(cv2.__version__))
print("Your Keras  version => {}".format(keras.__version__))

Your OpenCV version => 3.4.1
Your Keras  version => 2.1.5


In [ ]:
model = load_model('emotion_model.h5')

In [ ]:
moods = {
    0: 'ANGRY',
    1: 'FEAR',
    2: 'HAPPY',
    3: 'SAD',
    4: 'SURPRISE',
    5: 'NEUTRAL'}

In [ ]:
blanc_sq = np.zeros([48,48], dtype=np.uint8)


In [ ]:
# Loading the cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

# Defining a function that will do the detections
def detect(gray, frame):
    
    
    
    
    #     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, 
    minNeighbors=5, minSize=(30, 30),
    flags=cv2.CASCADE_SCALE_IMAGE)

    if len(faces) > 0:
        face = sorted(faces, reverse=True,
            key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (x, y, w, h) = face

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 0), 2)
        
        roi = gray[y:y + h, x:x + w]
        roi = cv2.resize(roi, (48, 48)).astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

    # make a prediction on the ROI, then lookup the class
    # label
        preds = model.predict(roi)[0]
        label = moods[preds.argmax()]
        #print(label)
        cv2.putText(frame,label, 
    (int(x), int(y-10)), 
     cv2.FONT_HERSHEY_SIMPLEX, 
    .6,
    (0,255,0),
    2)

    return frame


video_capture = cv2.VideoCapture(0)
while True:
    
    _, frame = video_capture.read()
    #frame = cv2.resize(frame, (640, 360))
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    canvas = detect(gray, frame)
    cv2.imshow('Video', canvas)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()